In [ ]:
pip install gradio

In [ ]:
pip install torch

In [ ]:
pip install sentencepiece

In [ ]:
pip install sacremoses

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification, MarianMTModel, MarianTokenizer

language_detection_model_name = "papluca/xlm-roberta-base-language-detection"
language_detection_tokenizer = AutoTokenizer.from_pretrained(language_detection_model_name)
language_detection_model = AutoModelForSequenceClassification.from_pretrained(language_detection_model_name)

base_translation_model = "Helsinki-NLP/opus-mt"
supported_languages = [
    {"code": "en", "name": "English"},
    {"code": "es", "name": "Spanish"},
    {"code": "fr", "name": "French"},
    {"code": "de", "name": "German"},
    {"code": "it", "name": "Italian"},
    {"code": "ja", "name": "Japanese"},
    {"code": "ru", "name": "Russian"},
    {"code": "ar", "name": "Arabic"},
    {"code": "nl", "name": "Dutch"},
    {"code": "hi", "name": "Hindi"},
]

translation_models = {}

#pair availability
for source_language in supported_languages:
    for target_language in supported_languages:
        if source_language["code"] != target_language["code"]:
            translation_model_name = f'{base_translation_model}-{source_language["code"]}-{target_language["code"]}'
            try:
                MarianMTModel.from_pretrained(translation_model_name)
                MarianTokenizer.from_pretrained(translation_model_name)
                translation_models[(source_language["code"], target_language["code"])] = translation_model_name
            except:
                print(f"Translation model not available for {source_language['name']} to {target_language['name']}")

def detect_and_translate(text, target_language_dropdown):
    # Language Detection
    inputs = language_detection_tokenizer(text, return_tensors="pt")
    outputs = language_detection_model(**inputs)
    predicted_language_id = outputs.logits.argmax().item()
    detected_language_code = list(language_detection_model.config.id2label.values())[predicted_language_id]
    detected_language = next(lang["name"] for lang in supported_languages if lang["code"] == detected_language_code)

    # Translation
    source_language_code = detected_language_code
    target_language_code = next(lang["code"] for lang in supported_languages if lang["name"] == target_language_dropdown)
    translation_model_name = translation_models.get((source_language_code, target_language_code))

    if translation_model_name is not None:
        try:
            translation_model = MarianMTModel.from_pretrained(translation_model_name)
            translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)

            translation_inputs = translation_tokenizer(text, return_tensors="pt")
            translation_outputs = translation_model.generate(**translation_inputs, max_length=50, num_beams=5, no_repeat_ngram_size=2)
            translated_text = translation_tokenizer.batch_decode(translation_outputs, skip_special_tokens=True)[0]

            return f"Detected language: {detected_language}\nTranslated to {target_language_dropdown}: {translated_text}"
        except Exception as e:
            print(f"Error during translation: {e}")
            return "Error during translation"
    else:
        return "Translation model not found for the specified language pair."

target_languages_dropdown = [lang["name"] for lang in supported_languages]

iface = gr.Interface(
    fn=detect_and_translate,
    inputs=[
        gr.Textbox(placeholder="Enter text for language detection and Translate...", label="Text"),
        gr.Dropdown(target_languages_dropdown, label="Select Language to be translated"),],
    outputs="text",
)
iface.launch()

Translation model not available for English to Japanese
Translation model not available for Spanish to Japanese
Translation model not available for Spanish to Hindi
Translation model not available for French to Italian
Translation model not available for French to Japanese
Translation model not available for French to Dutch
Translation model not available for French to Hindi
Translation model not available for German to Japanese
Translation model not available for German to Russian
Translation model not available for German to Hindi
Translation model not available for Italian to Japanese
Translation model not available for Italian to Russian
Translation model not available for Italian to Dutch
Translation model not available for Italian to Hindi
Translation model not available for Japanese to Hindi
Translation model not available for Russian to German
Translation model not available for Russian to Italian
Translation model not available for Russian to Japanese
Translation model not ava